In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd 
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno

from scipy import stats 

import warnings
warnings.filterwarnings('ignore')

In [ ]:
data = pd.read_csv("../input/ecommerce-data/data.csv", encoding="ISO-8859-1", dtype={'CustomerID': str})

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
data.describe(include='all').T

# **Missing Values**

In [ ]:
data.columns[data.isnull().any()]

In [ ]:
data_missings=data.filter(['Description', 'CustomerID'], axis=1)
msno.bar(data_missings);

In [ ]:
msno.heatmap(data); 

In [ ]:
def values_table(data_missings):
    mis_val = data_missings.isnull().sum()
    mis_val_percent = 100 * data_missings.isnull().sum() / len(data_missings)
    table = pd.concat([ mis_val,mis_val_percent], axis=1)
    table = table.rename(columns = {  0 :'Missing Values', 1 : '% Missing Value'})
    table['Data Type'] = data_missings.dtypes
    table = table[table.iloc[:,1] != 0].sort_values('% Missing Value', ascending=False).round(1)
    print ("There are " + str(data.shape[1]) + " columns and " + str(data.shape[0]) + " rows in the dataset.\n"      
             + str(table.shape[0]) + " of these columns have missing variables.")
    return table

values_table(data_missings) 

# 24.9% of customer information is missing. Customer information cannot be filled in any way. However, we can investigate the cause and effects of this missingness.

In [ ]:
data[data.Description.isnull()].CustomerID.isnull().value_counts(),data[data.Description.isnull()].UnitPrice.value_counts()

In [ ]:
data["Description_lower_case"] = data["Description"].str.lower()
data.Description_lower_case.dropna().value_counts()

In [ ]:
data[data["Description_lower_case"].str.contains("missing", na=False)]

In [ ]:
data[data["Description_lower_case"].str.contains("[?]", na=False)]

In [ ]:
data[data["Description_lower_case"].str.startswith("nan", na=False)]

In [ ]:
data.loc[data.Description_lower_case.isnull()==False, "Description_lower_case"] = data.loc[
    data.Description_lower_case.isnull()==False, "Description_lower_case"
].apply(lambda x: np.where("missing" in x, None, x))

In [ ]:
data[data["Description_lower_case"].str.contains("missing", na=False)]

In [ ]:
data.loc[data.Description_lower_case.isnull()==False, "Description_lower_case"] = data.loc[
    data.Description_lower_case.isnull()==False, "Description_lower_case"
].apply(lambda x: np.where("?" in x, None, x))

In [ ]:
data[data["Description_lower_case"].str.contains("[?]", na=False)]

In [ ]:
data.isnull().sum()

In [ ]:
df= data.dropna()

In [ ]:
df.isnull().sum()

In [ ]:
df=df.drop(['Description'], axis=1)

In [ ]:
df.duplicated().sum()

In [ ]:
df.drop_duplicates(inplace = True)

In [ ]:
df['InvoiceDate'] = pd.to_datetime(df.InvoiceDate, format='%m/%d/%Y %H:%M')
df['CustomerID'] = df['CustomerID'].astype('int64')

In [ ]:
df.insert(loc=2, column='Year', value=df.InvoiceDate.dt.year)
df.insert(loc=3, column='Month', value=df.InvoiceDate.dt.month)
df.insert(loc=4, column='Day', value=(df.InvoiceDate.dt.dayofweek)+1)

# Grouping Data

In [ ]:
df_1=df.groupby(['Country'], as_index=False)["InvoiceNo"].count()
df_1=df_1.sort_values(by='InvoiceNo', ascending=False).head(10)

In [ ]:
plt.figure(figsize=(10,5))
sns.barplot(df_1.Country, df_1.InvoiceNo, alpha=0.8,palette="Blues_r")
plt.title('Number of Transactions in Countries')
plt.ylabel('Number of Transactions', fontsize=12)
plt.xlabel('Country', fontsize=12)
plt.xticks(rotation=90)
plt.show()

In [ ]:
df_2=df.groupby(['Month','Year'], as_index=False)["InvoiceNo"].count()
plt.figure(figsize=(20,5))
sns.barplot(df_2.Month, df_2.InvoiceNo,palette="PiYG_r")
plt.ylabel("Number of Transactions")
plt.title("The Number of Transactions by Months")
plt.xticks([0,1,2,3,4,5,6,7,8,9,10,11],['Jan-2010','Feb-2011','Mar-2011','Apr-2011','May-2011','Jun-2011','July-2011','Aug-2011','Sep-2011','Oct-2011','Nov-2011','Dec-2010']);

In [ ]:
df_3=df.groupby(['Day'], as_index=False)["InvoiceNo"].count()
plt.figure(figsize=(20,5))
sns.barplot(df_3.Day, df_3.InvoiceNo,palette="Spectral_r")
plt.ylabel("Number of Transactions")
plt.title("The Number of Transactions by Days")
plt.xticks([0,1,2,3,4,5,6],['Mon','Tue','Wed','Thu','Fri','Sat','Sun']);

In [ ]:
df_4= df.groupby(['CustomerID', 'InvoiceNo'], as_index=False)['Quantity'].sum()
df_4=df_4.sort_values(by='CustomerID')
df_4

In [ ]:
df['Canceled_orders'] = df['InvoiceNo'].apply(lambda x:int('C' in x))
df_5=pd.DataFrame([{'Customers': len(df['CustomerID'].value_counts()),
               'Products': len(df['StockCode'].value_counts()),    
               'Orders': (df['Canceled_orders'].values == 0).sum(),
               'Canceled_orders' : (df['Canceled_orders'].values == 1).sum()}],index=['num.'])
df_5

In [ ]:
df_6=df[df['Canceled_orders'] == 1].groupby(['CustomerID', 'InvoiceNo','Canceled_orders'],
                                            as_index=False)['Quantity'].sum()
df_6.head()

In [ ]:
df['TotalPrice'] = df['UnitPrice'] * df['Quantity'] 
df_7= df.groupby(['CustomerID','InvoiceNo'], as_index=False)['TotalPrice'].sum()
df_7=df_7.sort_values(by='CustomerID')
df_7

In [ ]:
df_7=df[df.UnitPrice == 0].groupby(['CustomerID', 'StockCode','Description_lower_case'],
                                            as_index=False)['Quantity'].sum()
df_7.head()

In [ ]:
df_7=df[df.UnitPrice == 0].groupby(['CustomerID', 'InvoiceNo','Description_lower_case'],
                                            as_index=False)['Quantity'].sum()
df_7.head()

In [ ]:
df_8=df.groupby("StockCode").Description_lower_case.nunique()
df_8=df_8.sort_values(ascending=False).to_frame()
display(df_8.loc[df_8.Description_lower_case >1 ].head());
display(df.loc[df.StockCode == "23196"
              ].Description_lower_case.value_counts().to_frame());

# Outlier Handling

In [ ]:
df_outliers=df.filter([ 'Quantity', 'UnitPrice', 'Canceled_orders', 'TotalPrice'],axis=1)
plt.figure(figsize=(20,35))
for num,col in enumerate(df_outliers.columns,1):
    plt.subplot(8, 5, num)
    sns.boxplot(df_outliers[col])
    plt.tight_layout()

In [ ]:
df.loc[:, ["Quantity", "UnitPrice","TotalPrice"]].describe()

In [ ]:
df_new=df.select_dtypes(include=['int', 'float'])

outliers={}
for col in df_new:
    lower_lim = df_new[col].quantile(q=0.01)
    upper_lim = df_new[col].quantile(q=0.99)
    outliers[col] =  len(df_new[col][(df_new[col] > upper_lim)| (df_new[col] < lower_lim)])

In [ ]:
outliers_hardedge=pd.DataFrame(outliers.items(),columns=['Feature','Outliers'])
outliers_hardedge 

Too many outliers detected.It would not be right to remove it from the data immediately
because big data loss is not something we want. Let's try to visualize it as below.

In [ ]:
df_pozitive= df.loc[df.TotalPrice > 0].copy()
plt.figure(figsize=(18,5))
sns.distplot(np.log(df_pozitive.TotalPrice), bins=15, kde=False, color="blue");

In [ ]:
np.exp(-2.5),np.exp(8)

In [ ]:
plt.figure(figsize=(18,5))
sns.distplot(np.log(df.Quantity), bins=10, kde=False, color="blue");

In [ ]:
np.exp(0),np.exp(6.5)

In [ ]:
a=df["Quantity"].count()
b=df["Quantity"].loc[(df.Quantity <666 )].count()
c=a-b
a,b,c #We can extract 234 of them from the data

we can drop 234 of the quantity from the data

In [ ]:
m=df["TotalPrice"].count()
n=df["TotalPrice"].loc[(df.TotalPrice >0.08) & (df.TotalPrice <2981 ) ].count()
k=m-n
m,n,k

we can drop 8963 of the total price from the data.Again, it means a high number of outliers, so we can drop according to the hard edge method. But it is useful to do it after seeing the reaction of the model.

# Correlation

In [ ]:
df.corr()

In [ ]:
plt.figure(figsize=(15, 5))

heatmap = sns.heatmap(df.corr(), vmin=-1, vmax=1, annot=True)

heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':12}, pad=12);